In [4]:
import dt_apriltags as atag
import cv2
import numpy as np


# test images camera parameters
fx, fy, cx, cy = (
    867.1714530943214,
    7.2700778250832485,
    518.2694115721059,
    910.7793844438535,
)
fx, fy, cx, cy = (
    952., 952., 646., 342.
)
camera_params = [fx, fy, cx, cy]
k = [[fx,0,cx],[0,fy,cy],[0,0,1]]

In [5]:
def combine_rt(r, t):
    """Combine rotation and transformation matrices into one matrix

    :param r: 3x3 rotation matrix
    :type r: np.ndarray
    :param t: 3x1 translation matrix
    :type t: np.ndarray
    :return: 4x4 transformation matrix
    :rtype: np.ndarray
    """
    return np.array(
        [
            [r[0, 0], r[0, 1], r[0, 2], t[0, 0]],
            [r[1, 0], r[1, 1], r[1, 2], t[1, 0]],
            [r[2, 0], r[2, 1], r[2, 2], t[2, 0]],
            [0, 0, 0, 1],
        ]
    )


def scale_vec(origin, point, scale):
    diffs = point - origin
    length = np.linalg.norm(diffs)
    return origin + (diffs / length * scale)


def draw_pose(
    overlay,
    camera_params,
    tag_size,
    pose_r,
    pose_t,
    linescale=2,
    linewidth=5,
    z_sign=1,
    label=None,
):
    # def draw_pose(overlay, camera_params, tag_size, pose, width = 2, linescale = 2, z_sign=1):
    # fmt: off
    opoints = (np.array(
            [
                0,0,0,
                1,0,0,
                0,1,0,
                0,0,-1 * z_sign,
                0,-1 / linescale,0,
            ]
        ).reshape(-1, 1, 3)
        
        * 0.5
        * tag_size
        * linescale
    )
    boxpoints = (np.array(
            [
                -1, -1, 0,
                -1, 1, 0,
                1, 1, 0,
                1, -1, 0
            ]
        ).reshape(-1, 1, 3)
        * 0.5
        * tag_size
    )
    # fmt: on

    fx, fy, cx, cy = camera_params

    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

    # rvec, _ = cv2.Rodrigues(pose_r)
    # tvec = pose_t

    # rvec, _ = cv2.Rodrigues(pose[:3, :3])
    # tvec = pose[:3, 3]

    dcoeffs = np.zeros(5)

    ipoints, _ = cv2.projectPoints(opoints, pose_r, pose_t, K, dcoeffs)

    ipoints = np.round(ipoints).astype(int)
    ipoints = [tuple(pt) for pt in ipoints.reshape(-1, 2)]

    textsize = max(abs(ipoints[0][0] - ipoints[1][0]),abs(ipoints[0][0] - ipoints[2][0])) / 100

    dist = round(pose_t[2][0])


    # print(boxpoints)

    cv2.arrowedLine(overlay, ipoints[0], ipoints[1], (0, 0, 255), linewidth)
    cv2.arrowedLine(overlay, ipoints[0], ipoints[2], (0, 255, 0), linewidth)
    # cv2.arrowedLine(overlay, ipoints[0], ipoints[3], (255,0,0), linewdith)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(
        overlay,
        " X",
        ipoints[1],
        font,
        textsize,
        (0, 0, 255),
        round(linewidth / 2),
        cv2.LINE_AA,
    )
    cv2.putText(
        overlay,
        " Y",
        ipoints[2],
        font,
        textsize,
        (0, 255, 0),
        round(linewidth / 2),
        cv2.LINE_AA,
    )
    cv2.putText(
        overlay,
        str(dist),
        ipoints[0],
        font,
        textsize ,
        (255, 0, 0),
        round(linewidth / 2),
        cv2.LINE_AA,
    )
    if label:
        textwidth = (
            cv2.getTextSize(label, font, textsize / 35, round(linewidth / 2))[0][0] / 2
        )
        ori = (round(ipoints[4][0] - textwidth), ipoints[4][1])
        cv2.putText(
            overlay,
            label,
            ori,
            font,
            textsize ,
            (50, 255, 50),
            round(linewidth / 2),
            cv2.LINE_AA,
        )

    # _draw_cube(overlay,camera_params,tag_size,pose_r,pose_t,z_sign)



def get_tag_pixel_size(detection_object):
    corners = np.array(detection_object.corners)
    xdiffs = [
        abs(corners[0, 0] - corners[1, 0]),
        abs(corners[0, 0] - corners[2, 0]),
        abs(corners[3, 0] - corners[1, 0]),
        abs(corners[3, 0] - corners[2, 0]),
    ]
    ydiffs = [
        abs(corners[0, 1] - corners[1, 1]),
        abs(corners[0, 1] - corners[2, 1]),
        abs(corners[3, 1] - corners[1, 1]),
        abs(corners[3, 1] - corners[2, 1]),
    ]
    return max(max(xdiffs), max(ydiffs))

Coordinate System

The coordinate system has the origin at the camera center. The z-axis points from the camera center out the camera lens. The x-axis is to the right in the image taken by the camera, and y is down. The tag's coordinate frame is centered at the center of the tag, with x-axis to the right, y-axis down, and z-axis into the tag.

In [6]:
# read in image
# image = cv2.imread("testimages/3000.jpg")
image = cv2.imread("rs_cal_big/0.jpg")
# image = cv2.imread("testimages/cal1.jpg")
# image = cv2.imread("testimages/tag3blurred.jpg")
# image = cv2.imread("testimages/tag6d.jpg")
# image = cv2.imread("realsense/calnear_Color.png")
# image = cv2.imread("testimages/view300_300.jpg")

# image = cv2.blur(image,(10,10))

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# create detector
detector = atag.Detector(
    families="tagStandard41h12",
    #  quad_sigma=1,
    # max_hamming=5,
    # refine_edges=10
)

ft2m = 0.3048

size = 8
# size = 0.5

# convert outer edge size to tag size
size = size / 9 * 5

size = size *12 * ft2m


linewidth = 5
linescale = 2

# detect apriltag
results = detector.detect(gray, True, camera_params, size)

for tag in results:
    # print(tag)

    print("tag: ", tag.tag_id)

    p_size = get_tag_pixel_size(tag)

    corners = np.array(tag.corners, np.int32)
    corners = corners.reshape((-1, 1, 2))
    cv2.polylines(image, [corners], True, (255, 0, 0), linewidth)

    o = np.atleast_2d([[1], [1], [1]])
    r = np.array(tag.pose_R)
    t = np.array(tag.pose_t)
    # print(r)
    # print(t)

    d = np.linalg.norm(t)
    # print("d ", d, " mm")

    rvec, _ = cv2.Rodrigues(r)

    rt = combine_rt(r, t)

    print(r, t)

    draw_pose(
        image,
        camera_params,
        tag_size=size,
        pose_r=r,
        pose_t=t,
        linescale=linescale,
        linewidth=linewidth,
        z_sign=-1,
        label=str(tag.tag_id),
    )
cv2.imwrite("output.jpg", image)

print(f"Found {len(results)} tags. Results in output.jpg")

tag:  10
[[-0.0721824  -0.98522528  0.1553089 ]
 [ 0.98106743 -0.04207789  0.18904006]
 [-0.17971197  0.16601387  0.96960972]] [[ 62.36261006]
 [-15.57473935]
 [101.45955831]]
tag:  11
[[-0.01664714 -0.99150102  0.12902948]
 [ 0.9952558  -0.00405997  0.09720803]
 [-0.09585801  0.13003557  0.98686473]] [[58.65017881]
 [14.91615313]
 [93.68020337]]
tag:  12
[[-0.01240948 -0.99920808  0.03780492]
 [ 0.99508237 -0.00862489  0.09867464]
 [-0.09827044  0.03884351  0.99440138]] [[52.89879675]
 [43.08622179]
 [82.48536336]]
tag:  13
[[-0.10286519 -0.98254589  0.1549914 ]
 [ 0.94597011 -0.04846114  0.32061202]
 [-0.30750496  0.17959705  0.93444401]] [[ 18.26731975]
 [-15.50797017]
 [ 95.94239406]]
tag:  14
[[-0.07809741 -0.97904529  0.18807212]
 [ 0.94997597 -0.01585953  0.31192007]
 [-0.30240114  0.20302414  0.93130808]] [[15.61731385]
 [13.4953598 ]
 [90.86306354]]
tag:  15
[[-0.06175351 -0.99425976 -0.08737293]
 [ 0.99781092 -0.06357445  0.01821149]
 [-0.02366164 -0.08605704  0.99600919]] [[